## Import and define

In [1]:
import sys
cur_path = "/home/vmagent/app"
sys.path.append(cur_path)
from near_dedup import *


JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Configurate DIR

In [2]:
data_files = get_data_files('/home/vmagent/app/PILE/pile_uspto')
dup_dir = "/home/vmagent/app/PILE_output/pile_uspto/deduplicate"
ngram_size = 13
num_perm = 256
bands = 9
ranges = 13

## Load data into Spark

In [3]:
rdp = SparkDataProcessor()
spark=rdp.spark  

Will assign 48 cores and 308492 M memory for spark


23/08/30 16:26:53 WARN Utils: Your hostname, sr414 resolves to a loopback address: 127.0.1.1; using 10.1.2.14 instead (on interface enp134s0f1)
23/08/30 16:26:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/30 16:26:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


per core memory size is 6.276 GB and shuffle_disk maximum capacity is 8589934592.000 GB


In [4]:
with Timer("Load data with RowID"):
    df = read_json(data_files, spark).cache()
    total_length = df.count()

Load data with RowID started ...
/home/vmagent/app/PILE/pile_uspto/data_0_time1600242225_1976.jsonl
/home/vmagent/app/PILE/pile_uspto/data_0_time1600288933_1996.jsonl
/home/vmagent/app/PILE/pile_uspto/data_0_time1600364422_2006.jsonl
/home/vmagent/app/PILE/pile_uspto/data_10_time1600247352_1986.jsonl
/home/vmagent/app/PILE/pile_uspto/data_10_time1600397964_2016.jsonl
/home/vmagent/app/PILE/pile_uspto/data_11_time1600248055_1987.jsonl
/home/vmagent/app/PILE/pile_uspto/data_11_time1600403125_2017.jsonl
/home/vmagent/app/PILE/pile_uspto/data_12_time1600248735_1988.jsonl
/home/vmagent/app/PILE/pile_uspto/data_12_time1600407882_2018.jsonl
/home/vmagent/app/PILE/pile_uspto/data_13_time1600249593_1989.jsonl
/home/vmagent/app/PILE/pile_uspto/data_13_time1600413654_2019.jsonl
/home/vmagent/app/PILE/pile_uspto/data_14_time1600250414_1990.jsonl
/home/vmagent/app/PILE/pile_uspto/data_14_time1600418007_2020.jsonl
/home/vmagent/app/PILE/pile_uspto/data_15_time1600251300_1991.jsonl
/home/vmagent/app/

Load data with RowID took 212.13308584492188 sec


## Get minHashLSH edges

In [5]:
pipeline = minHashLSH_prepare(df, num_perm, ngram_size, bands, ranges)
with Timer("generate minHashLsh"):
    if os.path.exists(dup_dir):
        shutil.rmtree(dup_dir, ignore_errors=True)
    results = pipeline.saveAsTextFile(dup_dir)

num_bands is 9, ranges is 13
generate minHashLsh started ...


generate minHashLsh took 1390.2824745549588 sec


In [6]:
spark.stop()

## Generate connected components

In [7]:
with Timer(f"generate_connected_components all"):
    dup_connected_args = argparse.Namespace()
    dup_connected_args.input_dir = dup_dir
    dup_connected_args.out_file = os.path.join(
        dup_dir, "connected_components.pickle"
    )
    generate_connected_components.generate_connected_components_mp(dup_connected_args)

generate_connected_components all started ...
Started graph building


loop on file: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8801/8801 [00:01<00:00, 8729.32it/s]


length of the set of duplicates: 1220088 1.012829065322876


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1220088/1220088 [00:01<00:00, 706828.50it/s]


number of connected components: 613348 4.172353982925415
Graph generated duplicates list!!! 5.222286939620972
generate_connected_components all took 5.889061606954783 sec


##  convert as duplicates dict

In [8]:
with Timer(f"generate_duplicates_dict all"):
    dup_docs = os.path.join(dup_dir, "duplicates.pickle")
    dup_dict_args = argparse.Namespace()
    dup_dict_args.input_file = os.path.join(
        dup_dir, "connected_components.pickle"
    )
    dup_dict_args.out_file = dup_docs
    generate_duplicates_dict.generate_duplicates(dup_dict_args)

generate_duplicates_dict all started ...
Processing duplicates!!!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 613348/613348 [00:01<00:00, 553290.64it/s]


number of duplicate documents that will be removed: 1063551
generate_duplicates_dict all took 2.27969934605062 sec


## View result

In [9]:
dup_dict = pickle.load(open(os.path.join(dup_dir, "duplicates.pickle"), 'rb'))
dup_sum = 0
for _, v in dup_dict.items():
    dup_sum += len(list(v))

print(f"Completed!!")
print(f"    total processed {total_length} documents")
print(f"    total detected {dup_sum} duplicated documents")
print(f"    duplicate ratio is {dup_sum/total_length}")

Completed!!
    total processed 5883037 documents
    total detected 1063551 duplicated documents
    duplicate ratio is 0.18078264678600525
